In [1]:
import numpy as np
import os,re
import pandas as pd

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
# things
DATA_DIR = './data/amazon_raw.pkl'
EMBEDDINGS_DIR = './glove.840B.300d.txt'
MAX_SEQUENCE_LENGTH = 500
MAX_NUM_WORDS = 50000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.25

In [5]:
def clean_str(string):
    # remove stopwords
    # string = ' '.join([word for word in string.split() if word not in cachedStopWords])
    string = string.lower()
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [6]:
# load sentences
df = pd.read_pickle(DATA_DIR)

In [7]:
train_sentences = df[df['train/test']=='train']['text'].apply(clean_str).values

In [8]:
# build tockenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, lower=True)
tokenizer.fit_on_texts(train_sentences) # will take very long
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1332015 unique tokens.


In [9]:
# get sequence (padded)
x_train = tokenizer.texts_to_sequences(train_sentences)
x_train = pad_sequences(train_x, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

In [10]:
# get labels
mlb = MultiLabelBinarizer(sparse_output=True)
mlb.fit(df['categories'].values)
y_train = mlb.transform(df[df['train/test']=='train']['categories'].values)
labels_index = mlb.classes_

In [11]:
# get embeddings
vocab = set(tokenizer.word_index)
embeddings_index = {}
with open(EMBEDDINGS_DIR) as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        if word in vocab:
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 285061 word vectors.


In [12]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.random.uniform(-0.25, 0.25,(num_words, EMBEDDING_DIM))
# embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
empty = 0
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        empty+=1
print('{:.2f}% vocab has word embeddings'.format((num_words-empty)/num_words*100))

97.06% vocab has word embeddings


In [13]:
# set up embedding layer
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

W0628 18:05:25.093747 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [15]:
x_train = train_x
x_train.shape,y_train.shape

((1186239, 500), (1186239, 14286))

In [16]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Dense(128, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10)

W0628 18:07:34.142265 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 18:07:34.338004 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Training model.


W0628 18:07:34.354189 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0628 18:07:34.407079 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 18:07:34.421766 140030946133824 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0628 18:07:34.495368 140030946133824 deprecation.py:323] From /home/angela/env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
U

Epoch 1/10
1186239/1186239 [==============================] - 218s 184us/step - loss: 21.9291 - acc: 0.3101
Epoch 2/10
 755712/1186239 [==================>...........] - ETA: 1:14 - loss: 20.5505 - acc: 0.3083

KeyboardInterrupt: 